## This juypter notebook is tartget towards defining alleles 

The idea is that alleles are on corresponding haplotig and a good blast hit.

map all h contigs on their corresponding p alleles. Use the Assembletic '*oriented_coords.csv' file for this purpose.
This needs to be converted to a bed file or a gff file in order to pull out the overlap between gene models and the h_contig using either bedtools or pandas. Check what overlaps can spit out on the other side. 
>Considerations here are what happens if nothing overlaps or what if muliptle overlaps. 

Do gene blast of p on h with certain cut off. Pull out the best hit that is located on the corresponding haplotig. If that is not there look for best hit on other haplotigs or primary contigs that is not itself.

>Considerations are that p-h parings that are best hits and corresponding haplotig overule all others. There can only be a single hit. 



In [1]:
%matplotlib inline

In [6]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil


In [11]:
#Define the PATH
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
ASSEMBLETICS_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/nucmer_split/assemblytics'
BLAST_DB = os.path.join(BASE_AA_PATH, 'blast_DB')
OUT_PATH = os.path.join(BASE_AA_PATH, 'allele_analysis')
OUT_tmp = os.path.join(OUT_PATH, 'tmp')
if not os.path.isdir(OUT_PATH):
    os.mkdir(OUT_PATH)
if not os.path.isdir(OUT_tmp):
    os.mkdir(OUT_tmp)

In [12]:
#Define your p and h genome and move it into the allele analysis folder
p_genome = 'Pst_104E_v12_p_ctg'
h_genome = 'Pst_104E_v12_h_ctg'
genome_file_suffix = '.genome_file'
for x in (x + '.fa' for x in [p_genome, h_genome]):
    shutil.copy2(BASE_A_PATH+'/'+x, OUT_tmp)

In [28]:
{os.path.isfile(x + e) for e in ['.nin', '.nhr', '.nsq'] }

{True}

In [35]:
#generate the blast databases if not already present
os.chdir(BLAST_DB)
blast_dir_content = os.listdir(BLAST_DB)
for x in blast_dir_content:
    if x.endswith('.fa') and ({os.path.isfile(x + e) for e in ['.psq', '.phr', '.pin'] } != {True}\
           and {os.path.isfile(x + e) for e in ['.nin', '.nhr', '.nsq'] } != {True} ):

        make_DB_options = ['-in']
        make_DB_options.append(x)
        make_DB_options.append('-dbtype')
        if 'protein' in x:
            make_DB_options.append('prot')
        else:
            make_DB_options.append('nucl')
        make_DB_command = 'makeblastdb %s' % ' '.join(make_DB_options)
        make_DB_stderr = subprocess.check_output(make_DB_command, shell=True, stderr=subprocess.STDOUT)
        print('%s is done!' % make_DB_command)
print("All databases generated and ready to go!")

All databases generated and ready to go!


In [ ]:
make